In [79]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers, activations
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import cv2
from google.colab.patches import cv2_imshow
%load_ext tensorboard
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [80]:
datagen = ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 5,
    zoom_range = (0.95,0.95),
    horizontal_flip = False,
    vertical_flip = False,
    data_format = 'channels_last',
    validation_split = 0.0,
    dtype = 'float32',
)

train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/image_classifier/data/train',
    target_size = (100,100),
    batch_size = BATCH_SIZE,
    color_mode = 'rgb',
    class_mode = 'sparse',
    shuffle = True,
    subset = 'training'
)

test_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/image_classifier/data/test',
    target_size = (100,100),
    batch_size = BATCH_SIZE,
    color_mode = 'rgb',
    class_mode = 'sparse',
    shuffle = True,
    subset = 'training'
)

Found 160 images belonging to 2 classes.
Found 40 images belonging to 2 classes.


In [132]:
model = keras.models.Sequential(
    [
        keras.Input(shape = (100,100,3)),
        layers.Conv2D(filters = 32,kernel_size = 3,padding = 'valid',activation = 'relu',kernel_regularizer = regularizers.l2(0.01)),
        layers.MaxPooling2D(pool_size = (2,2)),
        layers.Conv2D(filters = 64,kernel_size = 3,padding = 'valid',activation = 'relu'),
        layers.MaxPooling2D(pool_size = (2,2)),
        layers.Conv2D(filters = 128,kernel_size = 3,padding = 'valid',activation = 'relu'),
        layers.Flatten(),
        layers.Dense(64, activation = 'relu'),
        layers.Dense(32, activation = 'relu'),
        layers.Dense(2)
    ]
)

callbacks = [
    keras.callbacks.BackupAndRestore("/content/drive/MyDrive/image_classifier/data/backup", save_freq="epoch", delete_checkpoint=True),
    keras.callbacks.EarlyStopping(
        monitor="accuracy",
        patience=5,
        verbose=2,
        mode="auto",
        start_from_epoch=0,
    ),
    keras.callbacks.TensorBoard(
        log_dir="logs",
        histogram_freq=1
    ),
    keras.callbacks.TerminateOnNaN(),
    keras.callbacks.ProgbarLogger()
]

model.compile(
    loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True),
    optimizer = keras.optimizers.Adam(learning_rate = 3e-4),
    metrics = ['accuracy']
)

model.fit(train_generator, epochs = 7, verbose = 2)
model.evaluate(test_generator)

class_names = ['LION','WOLF']
img = tf.keras.utils.load_img(
    '/content/drive/MyDrive/image_classifier/data/test/1/result1801051.jpeg', target_size=(100, 100)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0)

predictions = model.predict(img_array)
print(predictions)

print(f"This is a {class_names[np.argmax(predictions)]}")


Epoch 1/7
5/5 - 3s - loss: 0.7331 - accuracy: 0.5375 - 3s/epoch - 689ms/step
Epoch 2/7
5/5 - 1s - loss: 0.6252 - accuracy: 0.7750 - 882ms/epoch - 176ms/step
Epoch 3/7
5/5 - 1s - loss: 0.4866 - accuracy: 0.8062 - 880ms/epoch - 176ms/step
Epoch 4/7
5/5 - 1s - loss: 0.4414 - accuracy: 0.8062 - 898ms/epoch - 180ms/step
Epoch 5/7
5/5 - 1s - loss: 0.3837 - accuracy: 0.8562 - 902ms/epoch - 180ms/step
Epoch 6/7
5/5 - 1s - loss: 0.2958 - accuracy: 0.9125 - 869ms/epoch - 174ms/step
Epoch 7/7
5/5 - 1s - loss: 0.2536 - accuracy: 0.9250 - 889ms/epoch - 178ms/step
1/1 [==============================] - 0s 75ms/step
[[-197.51254  189.78888]]
This is a WOLF


In [ ]:
%tensorboard --logdir logs